# Basic stats on the evolution of COVID-19 hospitalizations in departments

In [1]:
%load_ext lab_black
%matplotlib inline

In [2]:
import matplotlib.pyplot as plt
from datetime import timedelta, date
import pandas as pd
import warnings

warnings.filterwarnings("ignore")
pd.plotting.register_matplotlib_converters()

In [3]:
covid_url = (
    "https://www.data.gouv.fr/fr/datasets/r/63352e38-d353-4b54-bfd1-f1b3ee1cabd7"
)
filter_dep = ["971", "972", "973", "974", "976"]  # only metropolitan France
figsize = (15, 10)

#### <br> Load table with COVID-19 data (figures by dates)

In [4]:
df = pd.read_csv(covid_url, sep=";", parse_dates=True, index_col=2)
df = df.query("sexe == 0")  # sum male/female
df.drop(columns=["sexe"], inplace=True)
df.query("dep not in @filter_dep", inplace=True)
df.sort_index(inplace=True)
df.head(2)

,dep,hosp,rea,rad,dc
jour,,,,,
2020-03-18,01,2,0,1,0
2020-03-18,02,41,10,18,11


In [5]:
last_day = df.index.max()
previous_day = last_day - timedelta(days=1)
print("Evolution from ", previous_day, " to ", last_day)

Evolution from  2020-04-15 00:00:00  to  2020-04-16 00:00:00


#### <br> Build a new dataframe with data from two latest days and the associated evolution

In [6]:
sdf = df.loc[last_day]
sdf.reset_index(inplace=True)
pdf = df.loc[previous_day]
pdf.reset_index(inplace=True)
frame = {"dep": sdf["dep"], "prev_hosp": pdf["hosp"], "last_hosp": sdf["hosp"]}
stats = pd.DataFrame(frame)
stats["diff"] = stats["last_hosp"] - stats["prev_hosp"]
stats["percent"] = stats["diff"] / stats["prev_hosp"] * 100
stats.head()

,dep,prev_hosp,last_hosp,diff,percent
0,01,139,153,14,10.071942
1,02,280,277,-3,-1.071429
2,03,59,60,1,1.694915
3,04,37,34,-3,-8.108108
4,05,44,42,-2,-4.545455


#### <br> Departments with highest number of hospitalizations on the latest day

In [7]:
stats.sort_values(by="last_hosp", ascending=False).head()

,dep,prev_hosp,last_hosp,diff,percent
75,75,3209,3103,-106,-3.303210
94,94,2278,2249,-29,-1.273047
92,92,1984,1977,-7,-0.352823
93,93,1574,1569,-5,-0.317662
69,69,1234,1228,-6,-0.486224


#### <br> Min, max, mean of evolution between the two latest days (in sum and %)

In [8]:
print(
    "Min: ",
    stats["diff"].min(),
    " Mean: ",
    stats["diff"].mean(),
    " Max: ",
    stats["diff"].max(),
)

Min:  -106  Mean:  -4.583333333333333  Max:  23


In [9]:
print(
    "Min %: ",
    stats.percent.min(),
    " Mean %: ",
    stats.percent.mean(),
    " Max %: ",
    stats.percent.max(),
)

Min %:  -16.666666666666664  Mean %:  -1.3400458113912224  Max %:  16.666666666666664


#### <br> List of departments with a drecrease of hospitalizations (last two days)

In [10]:
stats.query("diff < 0")

,dep,prev_hosp,last_hosp,diff,percent
1,02,280,277,-3,-1.071429
3,04,37,34,-3,-8.108108
4,05,44,42,-2,-4.545455
7,08,70,67,-3,-4.285714
9,10,266,263,-3,-1.127820
10,11,77,73,-4,-5.194805
12,13,1161,1147,-14,-1.205857
13,14,152,147,-5,-3.289474
15,16,13,11,-2,-15.384615
16,17,82,81,-1,-1.219512


In [11]:
print(stats.query("diff < 0").shape)

(60, 5)


#### <br> Departments with highest evolution of hospitalizations (total number)

In [12]:
stats.sort_values(by="diff", ascending=False).head()

,dep,prev_hosp,last_hosp,diff,percent
19,21,361,384,23,6.371191
62,62,471,491,20,4.246285
0,01,139,153,14,10.071942
53,53,78,91,13,16.666667
89,89,86,96,10,11.627907


#### <br> Departments with highest rate of hospitalizations (%)

In [13]:
stats.sort_values(by="percent", ascending=False).head()

,dep,prev_hosp,last_hosp,diff,percent
53,53,78,91,13,16.666667
58,58,55,63,8,14.545455
89,89,86,96,10,11.627907
0,01,139,153,14,10.071942
22,24,33,36,3,9.090909


#### <br> Focus on some departments

In [14]:
stats.query("dep=='75'")

,dep,prev_hosp,last_hosp,diff,percent
75,75,3209,3103,-106,-3.30321


In [15]:
stats.query("dep=='71'")

,dep,prev_hosp,last_hosp,diff,percent
71,71,187,190,3,1.604278


In [16]:
stats.query("dep=='31'")

,dep,prev_hosp,last_hosp,diff,percent
31,31,222,212,-10,-4.504505
